In [2]:
#research lab formulas task
import pandas as pd
from collections import defaultdict

In [3]:
#read_sheets
xl_file = pd.ExcelFile("batch2.xlsx")
advantage = 'hasFreeAccount'
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}
properties = []
#preprocessing
for df in dfs:
    dfs[df] = dfs[df].dropna()
    properties.append(df)    
dfs

{'healthy':                                              formula      result
 0  ltl formula1 {(healthy == true -> <>([]Custome...  fullfilled
 1  ltl formula2 {!(healthy == true -> <>([]Custom...  fullfilled
 2  ltl formula3 {(healthy == true -> <>([]Custome...  fullfilled
 3  ltl formula4 {!(healthy == true -> <>([]Custom...    violated,
 'income':                                              formula      result
 0  ltl formula1 {(income >= 2500) -> <>([]Custome...  fullfilled
 1  ltl formula2 {!(income >= 2500) -> <>([]Custom...    violated
 2  ltl formula3 {(income >= 2500)-> <>([]Customer...  fullfilled
 3  ltl formula4 {!(income >= 2500) -> <>([]Custom...  fullfilled}

In [4]:
def createDictionaries(df):
    df_length = int(len(df.index)/2)
    pair_dict = {}
    for i in range(df_length):
        pair = "p" + str(i+1);
        pair_dict[pair] = "";
        for index, row in df.iterrows():
            if i == 0:
                if index == i or index == i + 1:
                    pair_dict["p" + str(i+1)] = pair_dict["p" + str(i+1)] + " " + (df.iloc[index]['result'])
            elif i > 0 or i < df_length:
                if index == i + i or index == i + i + 1:
                    pair_dict["p" + str(i+1)] = pair_dict["p" + str(i+1)] + " " + (df.iloc[index]['result'])
    return pair_dict

In [5]:
results = {}
for df in dfs:
    dictionary = createDictionaries(dfs[df])
    results[df] = dictionary
results

{'healthy': {'p1': ' fullfilled fullfilled', 'p2': ' fullfilled violated'},
 'income': {'p1': ' fullfilled violated', 'p2': ' fullfilled fullfilled'}}

In [6]:
properties

['healthy', 'income']

In [7]:
final_result = {}
for prop in properties:
    for result in results:
        if prop == result:
            property_result= []
            for pairs in results[prop]:
                value = (results[prop][pairs]).split()
                if value[0] == value[1]:
                    property_result.append(value[0])
                else: 
#                     property_result.append("We have discrimination against"+ " " + prop)
                    property_result = ["We have discrimination against"+ " " + prop]
                    break;
        final_result[result] = property_result   
    
final_result

{'healthy': ['We have discrimination against healthy'],
 'income': ['We have discrimination against income']}

In [62]:
viol = 0;
notViol = 0;
for result in final_result:
    if len(final_result[result]) != 1:
        fullfilled_no = final_result[result].count('fullfilled')
        violated_no = final_result[result].count('violated')
        if fullfilled_no == 1 and violated_no >=1:
            final_result[result] = "No discrimination against"+ " " + result
            notViol+=1;
        elif fullfilled_no >= 1 and violated_no == 0:
            final_result[result] = "No discrimination against"+ " " + result
            notViol+=1;
        else:
            final_result[result] = "We have discrimination against"+ " " + result
            viol+=1;
    else:
        final_result[result] = final_result[result][0]
        
final_result

{'healthy': 'We have discrimination against healthy',
 'income': 'We have discrimination against income'}

In [70]:
text_file = open("result.html", "w")
header = ("<head><style>body{font-family: sans-serif; margin:20px 10%;display: flex;flex-flow: column;}"
"h1{color:#515355;font-size: 30px; text-transform: capitalize;width:100%;margin-bottom:0}"
"table{border: 1px solid; border-collapse: collapse; border-color: black;width: 100%}"
"table tr{text-align: left!important} td,th{padding: 10px} .no{font-size: 20px;text-transform: capitalize;width:100%}"
" .result{font-size:20px; font-style:italic;width: 100%;color: #2b475e;margin-bottom:40px} .red{color: red} .green{color:green}"
".note{font-size:20px; text-align:center} .counters{display:flex;flex-direction:column} "
".counters p{width:150px;display: flex;justify-content: space-between;align-items:center; font-weight: bold; margin-bottom:0}"
".counters p span{border: 2px solid;display: inline-block;width: 25px;height: 25px;border-radius: 50%;margin-left: 10px;text-align: center;line-height: 25px;}"
".top{display: flex;margin-bottom: 30px;justify-content: space-between;align-items: flex-end;}"
"a{font-style: italic;font-weight: bold;color: darkgoldenrod;}</style></head>")
#logo="<img src='IFD2.png' style='width:280px;margin-bottom: 0px;' />"
# counters="<div class='counters'><p style='color:darkred'>Violated<span>%s</span></p><p style='color:darkgreen'>Not Violated<span>%s</span></p></div>"%(viol, notViol)
topPart=("<div class='top'><img src='IFD3.png' style='width:280px;margin-bottom: 0px;' />"
"<div class='counters'><p style='color:darkred'>Violated<span>%s</span></p><p style='color:darkgreen'>Not Violated<span>%s</span></p></div></div>"%(viol, notViol))
text_file.write(header)
text_file.write(topPart)
# text_file.write(counters)
num = 0;
res = "";
for df in dfs:
    if final_result[df][0] == "W":
        res = "<span style='color:darkred'>not presereved</span>"
    else:
        res = "<span style='color:darkgreen'>presereved</span>"
    title = "<h1>%s</h1>" %(df) 
    table_no = "<p class='no'>Discrimination %s</p>" %(num + 1)
    result = ("<p class='result'>Based on information given in the table above, individual fairness is <span> %s with repect "
    "to '%s' when deciding about free account. This means that Free Account is dependent on %s</p>") %(res, df, df)
    table = dfs[df]
    html = table.to_html()
    text_file.write(title)
    text_file.write(table_no)
    text_file.write(html)
    text_file.write(result)
    num = num+1;

note=("<p class='note'>Developers/Readers who are interested in more details about discrimination, can take a look "
"<a href='details.html'>here</a></p>")
text_file.write(note)
text_file.close()


